In [ ]:
import numpy as np
import sys
from sklearn.cluster import KMeans
from numpy import random
from math import exp
from numpy.linalg import pinv

def createRandomDataPoints(totalDataPoints):
    # create random dataset as per specs
    inputDataPoints= np.empty((totalDataPoints,2))
    index=0
    for i in range(21):
        for j in range(21):
                xi = -2.0 + (0.2 * i)
                xj = -2.0 + (0.2 * j)
                inputDataPoints[index] = [xi,xj]
                index=index+1

    np.random.shuffle(inputDataPoints)
    return inputDataPoints

def outputMapping(totalDataPoints,dataPoints,num_of_classes):
    # Map output as per given specs
    dataPoint_index=-1
    output= np.empty((totalDataPoints,2))
    for i in range(len(dataPoints)):
        dataPoint_index = dataPoint_index +1
        dataPoint=dataPoints[i]
        if((dataPoint[0]**2 + dataPoint[1]**2) <= 1):
            output[dataPoint_index] = [1,0]
        else:
            output[dataPoint_index] = [0,1]
          
    output_array=np.array(output)
    return output_array

# Gaussian kernel function: h(x) = exp(-(abs(x - vi) ^ 2)/ (2 * width) ^ 2 )
# where x is the input vector and vi is the vector denoting the center of the radial function
def rbf_function(centerdataPoint,dataPoint,sigma_spread):
    centerdataPoint_arr=np.array(centerdataPoint)
    dataPoint_arr=np.array(dataPoint)
    numerator = float(((dataPoint_arr[0] - centerdataPoint_arr[0]) ** 2) + ((dataPoint_arr[1] - centerdataPoint_arr[1]) ** 2))
    g = float(np.exp(-numerator/(2*(sigma_spread ** 2))))
    return g

def calculate_Gaussian_Sum(dataPoints,centers,sigma_spread):

    dataPoint_index=0
    center_index=0
    rbf_data = [[0]*len(centers) for _ in range(len(dataPoints))]
  
    for i in range(len(centers)):
        for j in range(len(dataPoints)):
            rbf_data[dataPoint_index][center_index]=rbf_function(centers[i],dataPoints[j],sigma_spread)
            dataPoint_index = dataPoint_index + 1
        center_index = center_index + 1
        dataPoint_index=0

    rbf_array=np.array(rbf_data)

    return rbf_array


def mean_square_error(test_output,target_output):
    return np.square(np.sum(target_output - test_output, axis=1)).mean(axis=0)


def testing_accuracy(test_output,target_output):
    
    correctPrediction=0

    # make sure test output gets set to 0s and 1s.
    test_output=np.abs(np.round(test_output))

    for i in range(len(target_output)):
        targetPoint=target_output[i]
        predictedPoint=test_output[i]
        if ((targetPoint[0] == predictedPoint[0]) and (targetPoint[0] == predictedPoint[0])):
            correctPrediction=correctPrediction+1

    return correctPrediction / float(len(target_output)) * 100.0

def testNetwork(sigma_spread,center_selection):

    totalDataPoints=441
    trainDataPoints=[]
    testDataPoints=[]
    centers=[]

    trainDataPoints=createRandomDataPoints(totalDataPoints)

    if(center_selection==0):
        centers = trainDataPoints[:352,:2]
    elif(center_selection==1):
        random_indices=np.random.choice(range(len(trainDataPoints)), 150, replace=False)
        for i in range(150):
            centers.append(trainDataPoints[random_indices[i]])
    else:
        kmeans_data = KMeans(150, random_state=0).fit(trainDataPoints)
        centers=kmeans_data.cluster_centers_

    #First Train Network
    rbf_array=calculate_Gaussian_Sum(trainDataPoints,centers,sigma_spread)


    # weight_matrix =  pinv(G) . target_output
    G_plus = (np.linalg.pinv(rbf_array))
    target_output = outputMapping(totalDataPoints,trainDataPoints,2)
    weight_matrix = np.dot(G_plus,target_output)

    #Now get output using the weight_matrix
    testDataPoints=createRandomDataPoints(totalDataPoints)

    rbf_array2=calculate_Gaussian_Sum(testDataPoints,centers,sigma_spread)

    test_output=np.dot(rbf_array2,weight_matrix)
    test_output=np.array(test_output)

    rms_error=mean_square_error(test_output,target_output)
    accuracy_error=testing_accuracy(test_output,target_output)

    print(" %2f, %.15f, %.15f" %(sigma_spread,rms_error,accuracy_error))

def main():
    
    print("Using 352 data points as clusters")
    print("Sigma spread, Mean Square Error, testing_accuracy")
    for sigma_spread in np.arange(0.5,10.0, 0.2):
        testNetwork(sigma_spread,0)

    print("Using 150 random points as centers")
    print("Sigma spread, Mean Square Error, testing_accuracy")
    for sigma_spread in np.arange(0.5,10.0, 0.2):
        testNetwork(sigma_spread,1)

    print("Using KMeans algorithm for centers")
    print("Sigma spread, Mean Square Error, testing_accuracy")
    for sigma_spread in np.arange(0.5,10.0, 0.2):
        testNetwork(sigma_spread,2)


    sys.exit(0)

if __name__ == '__main__':
    main()

Using 352 data points as clusters
Sigma spread, Mean Square Error, testing_accuracy
 0.500000, 0.000000000018010, 71.428571428571431
 0.700000, 0.000011652683728, 68.934240362811792
 0.900000, 0.000007783208036, 69.614512471655331
 1.100000, 0.000003098533929, 72.789115646258509
 1.300000, 0.000001996957809, 73.242630385487530
 1.500000, 0.000000517333063, 72.335600907029473
 1.700000, 0.000001474608502, 71.428571428571431
 1.900000, 0.000000831405852, 70.068027210884352
 2.100000, 0.000000294179541, 75.056689342403629
 2.300000, 0.000000795462953, 71.428571428571431
 2.500000, 0.000001292354733, 72.789115646258509
 2.700000, 0.000000334261528, 71.655328798185948
 2.900000, 0.000000581674820, 70.521541950113374
 3.100000, 0.000000081438193, 73.242630385487530
 3.300000, 0.000001340217534, 72.789115646258509
 3.500000, 0.000000684527413, 71.428571428571431
 3.700000, 0.000002140470579, 70.975056689342409
 3.900000, 0.000000124829959, 72.789115646258509
 4.100000, 0.000000020429437, 72.7